In [2]:
import utils
import torch
import math
import numpy as np 
from PIL import Image
import cv2
import os
from pathlib import *
import shutil
import pandas as PD
import pillow_heif
import matplotlib.pyplot as plt
%matplotlib inline
PD.options.display.expand_frame_repr = False

In [3]:
display = utils.notebook_init() 
print('cuda доступна', torch.cuda.is_available())
print('под каким номером куда:',torch.cuda.current_device())
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" #без этого ошибка будет

YOLOv5  v6.1-258-g1156a32 Python-3.8.13 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)


Setup complete  (16 CPUs, 39.3 GB RAM, 232.9/953.9 GB disk)
cuda доступна True
под каким номером куда: 0


In [4]:
#Вот без этого ничего не работает.
#функция крутит картинку в зависимости от того, что нашла в характеристиках файла. 
def exif_transpose(img):
    if not img:
        return img
    wh = img.size
    if wh[0] > wh[1]:
        return img
    exif_orientation_tag = 274

    # Check for EXIF data (only present on some files)
    if hasattr(img, "_getexif") and isinstance(img._getexif(), dict) and exif_orientation_tag in img._getexif():
        exif_data = img._getexif()
        orientation = exif_data[exif_orientation_tag]

        # Handle EXIF Orientation
        if orientation == 1:
            # Normal image - nothing to do!
            pass
        elif orientation == 2:
            # Mirrored left to right
            img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 3:
            # Rotated 180 degrees
            img = img.rotate(180)
        elif orientation == 4:
            # Mirrored top to bottom
            img = img.rotate(180).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 5:
            # Mirrored along top-left diagonal
            img = img.rotate(-90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 6:
            # Rotated 90 degrees
            #print('повернули')
            img = img.rotate(-90, expand=True)
        elif orientation == 7:
            # Mirrored along top-right diagonal
            img = img.rotate(90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 8:
            # Rotated 270 degrees
            img = img.rotate(90, expand=True)

    return img

In [21]:
#model = torch.hub.load('ultralytics/yolov5', 'custom',  path='runs\\train\\bw\\exp4\\weights\\last.pt') #работает, но загружает каждый раз ЛУЧШЕЕ
model = torch.hub.load('ultralytics/yolov5', 'custom',  path='runs\\train\\yandex\\last.pt') #работает, но загружает каждый раз

Using cache found in C:\Users\Алексей/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  v6.1-258-g1156a32 Python-3.8.13 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 290 layers, 20852934 parameters, 0 gradients
Adding AutoShape... 


# Вариант 2. По фокусному расстоянию

In [6]:
def get_grayscale2(image):  #уменьшает изображение в 3 раза, что и хотелось бы сделать
    #https://habr.com/ru/post/163663/
    #convert image -colorspace gray image
    img = Image.open(image).convert('L')
    img.save(image)

In [7]:
from exif import Image as exif
def  get_focus_from_exif(f):
    with open(f, "rb") as palm_1_file:
        palm_1_image = exif(palm_1_file)
    if palm_1_image.has_exif:
        focal = palm_1_image.get('focal_length_in_35mm_film', 'Unknown')
        digital_zoom = palm_1_image.get('digital_zoom_ratio', 'Unknown')
        
         
    else:
        focal = 0
        digital_zoom = 1
    
    return focal, digital_zoom

In [8]:
def conv_heic_to_jpg(file, new_name):

    heif_file = pillow_heif.read_heif(file)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
    )
    
    #new_name = 'Y:\\temp.jpg' #так просто мы меняем расширение
    ex = heif_file.info['exif']
    image.save(new_name, format("jpeg"), exif=ex)
    image.close()
    return new_name

In [9]:
def get_distance(p):
    if len(p) != 2: 
        return 0
    d = (p[0][0]-p[1][0])**2 + (p[0][1]-p[1][1])**2
    d = d**0.5
    return d
def res(file, show_pandas = 0):
    if '.heic' in file:
        
        #name_f =file[file.rfind('\\')+1:]
        #name_f = name_f.replace('.heic','.jpg')
        temp_file = 'Y:\\tmp.jpg'
        file = conv_heic_to_jpg(work_file,temp_file)
    
    im = Image.open(file)
    (width, height) = im.size
    
    results = model(file)
    pd = results.pandas().xyxy[0]
    
    pd_point = pd.loc[pd['name'] == 'point']            #таблица с точками
    pd = pd.loc[pd['name'] == 'znak']                   #таблица со знаками
    
    #все номера нам не надо. Нам надо только тот, что ближе к центру

    #ОБРАБАТЫВАЕМ ЗНАКИ
    if show_pandas == 1:
        print('Знаки\n',pd)
        print('Точки\n',pd_point)
    
    pd = pd.assign(to_centre_x = abs(width/2 - (pd.xmin + (pd.xmax-pd.xmin)/2)))     #считаем расстояние до центра
    pd = pd.assign(centre_x = pd.xmin + (pd.xmax-pd.xmin)/2)                         #считаем координаты центра 
    
    pd = pd.assign(width = pd.xmax - pd.xmin)                                        #ширина
    pd = pd.assign(height = pd.ymax - pd.ymin)                                       #высота
    pd = pd.assign(s2 = pd.width * pd.height)                                        #площадь
    
    pd = pd.sort_values(['to_centre_x'] )                                            #сортируем по близости к центру. Нам надо самый близкий номер
    
    
    d = dict()   #в этом словаре будут все результаты
    for index, row in pd.iterrows():
        d ['width']  = row['width']
        d ['height']  = row['height']
        d ['s2']  = row['s2']
        d ['xmin']  = row['xmin']
        d ['ymin']  = row['ymin']
        d ['xmax']  = row['xmax']
        d ['ymax']  = row['ymax']
        break
    #Сейчас в d - все характеристики знака. 
    
    
    #ОБРАБАТЫВАЕМ POINTS. Берем только те, что входят в область нашего знака. В идеале их должно быть две.
    pd_point = pd_point.assign(centre_x = pd_point.xmin + (pd_point.xmax-pd_point.xmin)/2)   #Yнам нужны будут только центры точек
    pd_point = pd_point.assign(centre_y = pd_point.ymin + (pd_point.ymax-pd_point.ymin)/2)   #Yнам нужны будут только центры точек
    
    
    pd_point = pd_point.sort_values(['centre_x'] )
    if show_pandas == 1:
        print('все найденные точки\n',pd_point)
    points_in_znak =[]
    for index, row in pd_point.iterrows():
        if (d['xmin'] < row['centre_x'] < d['xmax']) and (d['ymin'] < row['centre_y'] < d['ymax']):
            points_in_znak.append([row['centre_x'],row['centre_y']])
    
    if show_pandas == 1:
        print('координаты точек в знаке', points_in_znak)
    
    d['main_line'] = get_distance(points_in_znak)
    
    d['im_width'] = width
    d['im_height'] = height
    
    
    return d    
        
             

In [10]:
def crop_znak(temp_file,znak):
    if len(znak) == 3:
        print('не смог кроопнуть', temp_file)
        return
    img = Image.open(temp_file)
    xmin  = znak['xmin']
    ymin = znak['ymin']
    xmax  = znak['xmax']
    ymax  = znak['ymax']
    #print((xmin, ymin,xmax, ymax))
    img = img.crop((xmin, ymin,xmax, ymax))
    img.save(temp_file)


In [27]:
#пытаемся вырезать область вокруг знака, чтобы подать ее потом снова в сеть для более точного распознавания.
def mini_pic(temp_file,znak):
    if len(znak) == 3:
        print('не смог кроопнуть', temp_file)
        return
    if znak['width'] >400:
        return
    
    img = Image.open(temp_file)
    xmin  = znak['xmin']
    ymin = znak['ymin']
    xmax  = znak['xmax']
    ymax  = znak['ymax']
    centre_x = xmin + (xmax - xmin)/2
    centre_y = ymin + (ymax - ymin)/2
    #print((xmin, ymin,xmax, ymax))
    img = img.crop((centre_x-480, centre_y - 480,centre_x+480, centre_y + 480))
    img.save(temp_file)
    print('минипик',temp_file,centre_x,centre_y, znak['width'])
    
#пытаемся вырезать область вокруг знака, чтобы подать ее потом снова в сеть для более точного распознавания.
def zoom960(temp_file):
    
    
    img = Image.open(temp_file)
    w,h = img.size
    centre_x = w//2
    centre_y = h//2
    #print((xmin, ymin,xmax, ymax))
    img = img.crop((centre_x-480, centre_y - 480,centre_x+480, centre_y + 480))
    img.save(temp_file)
    #print('минипик',temp_file,centre_x,centre_y, znak['width'])

In [12]:
from math import atan, sin, cos, tan,degrees

def correct_znak2(znak):
    
    w_znak = 520 #ширина знака в мм
    h_znak = 112
    if len(znak) == 3: #похоже, что не рааспознался
        return w_znak
    #сначала ищем угол наклона знака по соотношению сторон
    HW  = 112/520 #стандартное соотношение между сторонами
    hw = znak['height']/znak['width']#найденное соотнощение между сторонами
    tan_alfa = -(HW - hw) / (1 - HW * hw)
    alfa_rad = abs(math.atan(tan_alfa)) #нашли угол наклона знака в радианах
    print('приблизительно повернуто',degrees(alfa_rad))
    
    new_AB = w_znak * cos(alfa_rad) + h_znak * sin(alfa_rad)
    #new_w = (CD-AB/tan(alfa_rad)) / (sin(alfa_rad) - cos(alfa_rad)*cos(alfa_rad)/sin(alfa_rad))
    
    print('корректировка длины знака','520', '---->',new_AB)
    
    return new_AB

def correct_znak(znak):
    if len(znak) == 3: #похоже, что не рааспознался
        return znak
    #сначала ищем угол наклона знака по соотношению сторон
    HW  = 112/520 #стандартное соотношение между сторонами
    hw = znak['height']/znak['width']#найденное соотнощение между сторонами
    tan_alfa = -(HW - hw) / (1 - HW * hw)
    alfa_rad = math.atan(tan_alfa) #нашли угол наклона знака в радианах
    print('приблизительно повернуто',degrees(alfa_rad))
    
    CD = znak['height']
    AB = znak['width']
    new_w = (CD-AB/tan(alfa_rad)) / (sin(alfa_rad) - cos(alfa_rad)*cos(alfa_rad)/sin(alfa_rad))
    
    print('корректировка длины знака',znak['width'], '---->',new_w)
    znak['width'] = new_w
    return znak

def correct_zoom(znak,digital_zoom):
    if digital_zoom != 0:
        new_w = znak['width'] / digital_zoom
        znak['width'] = new_w
    return znak
    

In [30]:
def file_recognition(f):
    temp_path = 'Y:\\'
    w_znak = 520 #ширина знака в мм
    w = 4032    #разрешение матрицы/фото в пикселях
    w_matrix = 34.974 #ширина матрицы в мм
    pixel_in_mm = w/w_matrix
    name_file = f[f.rfind('\\')+1:]
    
    
    
    work_file = f
    temp_file = os.path.join(temp_path, name_file)
    
    
    if '.heic' in f:
        temp_file = temp_file.replace('.heic','.jpg')
        work_file = conv_heic_to_jpg(work_file,temp_file)
        print('Сконвертирован',work_file)
    focus, digital_zoom = get_focus_from_exif(work_file)
    print('focus, digital_zoom',focus, digital_zoom)
     
    img = Image.open(work_file)
    img = exif_transpose(img)
    #print('сохранили поворот',temp_file)
    img.save(temp_file)
    img.close()

    #get_grayscale2(temp_file)
    
    znak1 = res(temp_file)
    znak = znak1
    
    #Заходим еще 1 раз
    if len(znak) == 3:
        zoom960(temp_file)
        znak1 = res(temp_file)
        znak = znak1
        
    
    
    #ЭКСПЕРИМЕНТ
#     if len(znak1) == 3:
#         print('ФАЙЛ НЕ РАСПОЗНАН', f)
#         not_recognize.append(f) 
#         return 0.0
    
#     #пробуем вырезать область,чтобы знак был бллиже. количество пикселей от этого не меняется
#     print('ширина до мини пика', znak1['width'])

#     mini_pic(temp_file,znak1)
#     #вырезали и еще раз на распознавание
#     znak = res(temp_file)
#     if len(znak) != 3:
#         print('ширина после мини пика', znak['width'], 'уточнение', znak1['width']-znak['width'])
#     else: #почему то после зума, знак не распознался. Возвращаем то, что было, это всяко лучше чем ноль
#         znak = znak1
        
    #....конец эксперимента.
    
    
    #crop_znak(temp_file,znak)
    
    
    #кооректировка длины знака
    #znak = correct_znak(znak) #корректируем длину знака в зависимости от наклона
    
    #уточнение полученнного расстояние
    w_znak = correct_znak2(znak)
    
    #znak = correct_zoom(znak,digital_zoom) #корректируем длину знака взависимости от цифрового увеличения
    #pixel_in_mm = pixel_in_mm / digital_zoom
    
    if len(znak) == 3:
        print('ФАЙЛ НЕ РАСПОЗНАН', f)
        not_recognize.append(f)
        d = 0.0
    #формируем строку с параметрами знака для записи в csv
    else:
        d = (znak['width']/pixel_in_mm + w_znak)/(znak['width']/pixel_in_mm) * focus/1000
        #d = (w_znak)/(znak['width']/pixel_in_mm) * focus/1000
    
    return d

In [31]:
train_csv = '..\\avto\\start_dataset\\train.csv'
#сначала прочитаем все данные нам растояния
dist = dict()  #здесь будем храть все расстояния добытые из файла
with open(train_csv, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.replace('\n','')
        key, d = line.split(';')
        dist[key] = d
        
        

keys = ['width', 'height', 's2', 'xmin', 'ymin', 'xmax', 'ymax', 'main_line', 'im_width', 'im_height']
pic_data = '..\\avto\\start_dataset\\test'
train_csv = '..\\avto\\start_dataset\\my_train.csv'
test = '..\\avto\\start_dataset\\sample_solution.csv'
temp_path = 'Y:\\'


new_f = 'image_name;distance'
#print(new_f)
not_recognize = []
n = 0
abs_mistake, otn_mistake, vsego = 0, 0, 0
for f in os.listdir(pic_data):
    n +=1
#     if n<= 400:
#         continue
#     if n>5:
#         break
    
    
    work_file = os.path.join(pic_data, f)
    print(n,'работаем  с файлом: ',work_file)

    itog = file_recognition(work_file)
    
    if f in dist:
        y = float(dist[f])
        mistake = (y - itog) / y
        print('ошибка',mistake, 'ист', y, 'предск',itog)
        otn_mistake += mistake
        abs_mistake += mistake
        vsego +=1
    
    
    st = f+';' + str(itog)
    print('ВЫХОД',st)
    print('\n\n')    
    new_f = new_f +'\n'+st
    
#print(new_f)
with open(test, 'w', encoding = 'utf-8') as file:
    file.write(new_f)
print('не распознано файлов', len(not_recognize))
print('не распознанные', *not_recognize)
print('сохранено в ',test)
if vsego != 0:
    print('Абс ошибка', round(abs_mistake/vsego*100, 2))
    print('Относительная ошибка', round(otn_mistake/vsego*100, 2))
else:
    print('Ошибка не посчитана. Истинных данных не обнаружено. Датасет тестовый?')

1 работаем  с файлом:  ..\avto\start_dataset\test\img_1597.jpg
focus, digital_zoom 17 1.261050875729775
приблизительно повернуто 2.718017245800112
корректировка длины знака 520 ----> 524.7261091524025
ВЫХОД img_1597.jpg;3.698082365227473



2 работаем  с файлом:  ..\avto\start_dataset\test\img_1598.jpg
focus, digital_zoom 17 1.261050875729775
приблизительно повернуто 2.86107512838224
корректировка длины знака 520 ----> 524.9422352093543
ВЫХОД img_1598.jpg;2.619093093641067



3 работаем  с файлом:  ..\avto\start_dataset\test\img_1599.jpg
focus, digital_zoom 17 1.261050875729775
приблизительно повернуто 3.795022592369584
корректировка длины знака 520 ----> 526.2727239061583
ВЫХОД img_1599.jpg;1.9383201604253346



4 работаем  с файлом:  ..\avto\start_dataset\test\img_1602.jpg
focus, digital_zoom 17 1.261050875729775
приблизительно повернуто 0.18992456131367969
корректировка длины знака 520 ----> 520.3684010532226
ВЫХОД img_1602.jpg;2.6158998714055914



5 работаем  с файлом:  ..\avto\st

приблизительно повернуто 5.408067754066908
корректировка длины знака 520 ----> 528.24115738334
ВЫХОД img_1682.jpg;8.70298314723439



36 работаем  с файлом:  ..\avto\start_dataset\test\img_1683.jpg
focus, digital_zoom 16 1.237315875613748
приблизительно повернуто 1.143226317939095
корректировка длины знака 520 ----> 522.1310858156323
ВЫХОД img_1683.jpg;8.471667717222001



37 работаем  с файлом:  ..\avto\start_dataset\test\img_1689.jpg
focus, digital_zoom 16 1.237315875613748
приблизительно повернуто 1.924635366714045
корректировка длины знака 520 ----> 523.4681609032036
ВЫХОД img_1689.jpg;3.6296354311143535



38 работаем  с файлом:  ..\avto\start_dataset\test\img_1693.jpg
focus, digital_zoom 16 1.237315875613748
приблизительно повернуто 3.2308615227257227
корректировка длины знака 520 ----> 525.4857277236526
ВЫХОД img_1693.jpg;3.657756395513506



39 работаем  с файлом:  ..\avto\start_dataset\test\img_1694.jpg
focus, digital_zoom 16 1.237315875613748
приблизительно повернуто 2.712498

приблизительно повернуто 0.7070113573309322
корректировка длины знака 520 ----> 521.3424195678509
ВЫХОД img_1844.jpg;3.5771146027370406



70 работаем  с файлом:  ..\avto\start_dataset\test\img_1846.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 3.0718020764973364
корректировка длины знака 520 ----> 525.254631895661
ВЫХОД img_1846.jpg;5.0178500450967825



71 работаем  с файлом:  ..\avto\start_dataset\test\img_1849.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 3.8347991859366934
корректировка длины знака 520 ----> 526.326286979762
ВЫХОД img_1849.jpg;4.752361317932851



72 работаем  с файлом:  ..\avto\start_dataset\test\img_1851.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 2.8353206443114405
корректировка длины знака 520 ----> 524.9035678845004
ВЫХОД img_1851.jpg;3.440885993457416



73 работаем  с файлом:  ..\avto\start_dataset\test\img_1852.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто

приблизительно повернуто 4.731765233407441
корректировка длины знака 520 ----> 527.4667357157319
ВЫХОД img_1908.jpg;5.808219038550929



104 работаем  с файлом:  ..\avto\start_dataset\test\img_1911.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 2.602038130244317
корректировка длины знака 520 ----> 524.5484915006668
ВЫХОД img_1911.jpg;4.155146120527584



105 работаем  с файлом:  ..\avto\start_dataset\test\img_1913.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 1.7601266888921288
корректировка длины знака 520 ----> 523.1947519447757
ВЫХОД img_1913.jpg;2.95796288335996



106 работаем  с файлом:  ..\avto\start_dataset\test\img_1914.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 1.609312776620729
корректировка длины знака 520 ----> 522.9403137385444
ВЫХОД img_1914.jpg;2.3857907389449338



107 работаем  с файлом:  ..\avto\start_dataset\test\img_1915.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернут

приблизительно повернуто 2.077812545126926
корректировка длины знака 520 ----> 523.7188574870426
ВЫХОД img_1975.jpg;1.4187530741283445



138 работаем  с файлом:  ..\avto\start_dataset\test\img_1978.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 2.732043630764163
корректировка длины знака 520 ----> 524.7474443122954
ВЫХОД img_1978.jpg;2.7407940579887056



139 работаем  с файлом:  ..\avto\start_dataset\test\img_1981.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 1.2155561010345362
корректировка длины знака 520 ----> 522.2589323893449
ВЫХОД img_1981.jpg;2.389959754696938



140 работаем  с файлом:  ..\avto\start_dataset\test\img_1982.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 1.4308875574300202
корректировка длины знака 520 ----> 522.6346137211127
ВЫХОД img_1982.jpg;3.0005853789393435



141 работаем  с файлом:  ..\avto\start_dataset\test\img_1986.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно пове

...

приблизительно повернуто 0.8666155988668908
корректировка длины знака 520 ----> 521.634488257279
ВЫХОД img_2655.jpg;5.1142702712129156



376 работаем  с файлом:  ..\avto\start_dataset\test\img_2661.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 2.6312425366705114
корректировка длины знака 520 ----> 524.5934194448962
ВЫХОД img_2661.jpg;4.770436113429348



377 работаем  с файлом:  ..\avto\start_dataset\test\img_2662.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 1.4393899944155963
корректировка длины знака 520 ----> 522.649296170182
ВЫХОД img_2662.jpg;4.182593125238208



378 работаем  с файлом:  ..\avto\start_dataset\test\img_2663.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 2.235382171506361
корректировка длины знака 520 ----> 523.9728372020693
ВЫХОД img_2663.jpg;3.9158342757942495



379 работаем  с файлом:  ..\avto\start_dataset\test\img_2668.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно поверн

приблизительно повернуто 1.9250377040378772
корректировка длины знака 520 ----> 523.4688242878414
ВЫХОД img_2701.jpg;2.062462227796753



409 работаем  с файлом:  ..\avto\start_dataset\test\img_2702.heic
Сконвертирован Y:\img_2702.jpg
focus, digital_zoom 14 1.0327868852459017
приблизительно повернуто 0.525613736661571
корректировка длины знака 520 ----> 521.005558341416
ВЫХОД img_2702.heic;3.440649720707308



410 работаем  с файлом:  ..\avto\start_dataset\test\img_2702.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 0.5752186885525342
корректировка длины знака 520 ----> 521.0981952584982
ВЫХОД img_2702.jpg;1.4222406358561914



411 работаем  с файлом:  ..\avto\start_dataset\test\img_2703.heic
Сконвертирован Y:\img_2703.jpg
focus, digital_zoom 14 1.0327868852459017
приблизительно повернуто 1.6448210287010392
корректировка длины знака 520 ----> 523.0005458935981
ВЫХОД img_2703.heic;2.819214550351693



412 работаем  с файлом:  ..\avto\start_dataset\test\img_2703.j

приблизительно повернуто 1.0028841204205663
корректировка длины знака 520 ----> 521.8806504464279
ВЫХОД img_2729.jpg;2.6398960969159644



441 работаем  с файлом:  ..\avto\start_dataset\test\img_2731.heic
Сконвертирован Y:\img_2731.jpg
focus, digital_zoom 14 1.0327868852459017
приблизительно повернуто 2.2757147613107773
корректировка длины знака 520 ----> 524.0372106468334
ВЫХОД img_2731.heic;5.711180325623725



442 работаем  с файлом:  ..\avto\start_dataset\test\img_2731.jpg
focus, digital_zoom 14 1.0595655220742817
приблизительно повернуто 1.2711159794886138
корректировка длины знака 520 ----> 522.3565721068668
ВЫХОД img_2731.jpg;3.848425029526305



443 работаем  с файлом:  ..\avto\start_dataset\test\img_2732.heic
Сконвертирован Y:\img_2732.jpg
focus, digital_zoom 14 1.0327868852459017
приблизительно повернуто 2.7030267425686447
корректировка длины знака 520 ----> 524.7032727343942
ВЫХОД img_2732.heic;4.9190382542544295



444 работаем  с файлом:  ..\avto\start_dataset\test\img_273

приблизительно повернуто 2.0232559082608548
корректировка длины знака 520 ----> 523.6299967542626
ВЫХОД img_2776.jpg;4.8807071712470815



474 работаем  с файлом:  ..\avto\start_dataset\test\img_2777.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 0.749929040312099
корректировка длины знака 520 ----> 521.4213547740031
ВЫХОД img_2777.jpg;5.409926588438739



475 работаем  с файлом:  ..\avto\start_dataset\test\img_2778.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 0.017315196896913174
корректировка длины знака 520 ----> 520.0338234599017
ВЫХОД img_2778.jpg;6.032548548243174



476 работаем  с файлом:  ..\avto\start_dataset\test\img_2781.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 0.3378905965244958
корректировка длины знака 520 ----> 520.6514518520785
ВЫХОД img_2781.jpg;7.14730885421625



477 работаем  с файлом:  ..\avto\start_dataset\test\img_2782.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно пове

приблизительно повернуто 1.7592745907422531
корректировка длины знака 520 ----> 523.193324550617
ВЫХОД img_2917.jpg;2.88546328494917



508 работаем  с файлом:  ..\avto\start_dataset\test\img_2919.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 1.0774570274067377
корректировка длины знака 520 ----> 522.0141129402298
ВЫХОД img_2919.jpg;1.8230440394855587



509 работаем  с файлом:  ..\avto\start_dataset\test\img_2921.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 2.3430228305713494
корректировка длины знака 520 ----> 524.1440604314473
ВЫХОД img_2921.jpg;2.561946413968575



510 работаем  с файлом:  ..\avto\start_dataset\test\img_2922.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 0.022941642671602105
корректировка длины знака 520 ----> 520.0448039205102
ВЫХОД img_2922.jpg;2.98279375049508



511 работаем  с файлом:  ..\avto\start_dataset\test\img_2923.jpg
focus, digital_zoom 14 1.0223123732251522
приблизительно повер

In [89]:
itog = file_recognition('..\\avto\\start_dataset\\test\\img_1710.jpg')
itog = file_recognition('..\\avto\\start_dataset\\test\\img_2674.heic')
itog = file_recognition('..\\avto\\start_dataset\\test\\img_2715.jpg')

focus, digital_zoom 16 1.237315875613748
ФАЙЛ НЕ РАСПОЗНАН ..\avto\start_dataset\test\img_1710.jpg
Сконвертирован Y:\img_2674.jpg
focus, digital_zoom 14 1.0327868852459017
ФАЙЛ НЕ РАСПОЗНАН ..\avto\start_dataset\test\img_2674.heic
focus, digital_zoom 14 1.0223123732251522
приблизительно повернуто 0.7092206576181656
корректировка длины знака 520 ----> 521.3464901064322


In [26]:
img = Image.open('..\\avto\\start_dataset\\test\\img_1710.jpg')
w,h = img.size
print(w,h)

4032 3024


# Делаем  датасет ч/б

In [113]:
def get_grayscale(image):  #это не уменьшает почему то размер изображения
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [8]:
n = 0
pic_data = '..\\avto\\train_data3\\images\\train\\'
for f in os.listdir(pic_data):
    n +=1
    work_file = os.path.join(pic_data, f)
    print(n,'работаем  с файлом: ',work_file)
    #img = cv2.imread(work_file)
    img = get_grayscale2(work_file)
    #cv2.imwrite(work_file,img)
print('ОК')

1 работаем  с файлом:  ..\avto\train_data3\images\train\000000000009.jpg
2 работаем  с файлом:  ..\avto\train_data3\images\train\000000000025.jpg
3 работаем  с файлом:  ..\avto\train_data3\images\train\000000000030.jpg
4 работаем  с файлом:  ..\avto\train_data3\images\train\000000000034.jpg
5 работаем  с файлом:  ..\avto\train_data3\images\train\000000000036.jpg
6 работаем  с файлом:  ..\avto\train_data3\images\train\000000000042.jpg
7 работаем  с файлом:  ..\avto\train_data3\images\train\000000000049.jpg
8 работаем  с файлом:  ..\avto\train_data3\images\train\000000000061.jpg
9 работаем  с файлом:  ..\avto\train_data3\images\train\000000000064.jpg
10 работаем  с файлом:  ..\avto\train_data3\images\train\000000000071.jpg
11 работаем  с файлом:  ..\avto\train_data3\images\train\000000000072.jpg
12 работаем  с файлом:  ..\avto\train_data3\images\train\000000000073.jpg
13 работаем  с файлом:  ..\avto\train_data3\images\train\000000000074.jpg
14 работаем  с файлом:  ..\avto\train_data3\ima

129 работаем  с файлом:  ..\avto\train_data3\images\train\img_1853.jpg
130 работаем  с файлом:  ..\avto\train_data3\images\train\img_1854.jpg
131 работаем  с файлом:  ..\avto\train_data3\images\train\img_1856.jpg
132 работаем  с файлом:  ..\avto\train_data3\images\train\img_1857.jpg
133 работаем  с файлом:  ..\avto\train_data3\images\train\img_1860.jpg
134 работаем  с файлом:  ..\avto\train_data3\images\train\img_1861.jpg
135 работаем  с файлом:  ..\avto\train_data3\images\train\img_1862.jpg
136 работаем  с файлом:  ..\avto\train_data3\images\train\img_1863.jpg
137 работаем  с файлом:  ..\avto\train_data3\images\train\img_1864.jpg
138 работаем  с файлом:  ..\avto\train_data3\images\train\img_1865.jpg
139 работаем  с файлом:  ..\avto\train_data3\images\train\img_1874.jpg
140 работаем  с файлом:  ..\avto\train_data3\images\train\img_1878.jpg
141 работаем  с файлом:  ..\avto\train_data3\images\train\img_1881.jpg
142 работаем  с файлом:  ..\avto\train_data3\images\train\img_1882.jpg
143 ра

245 работаем  с файлом:  ..\avto\train_data3\images\train\img_2305.jpg
246 работаем  с файлом:  ..\avto\train_data3\images\train\img_2306.jpg
247 работаем  с файлом:  ..\avto\train_data3\images\train\img_2308.jpg
248 работаем  с файлом:  ..\avto\train_data3\images\train\img_2309.jpg
249 работаем  с файлом:  ..\avto\train_data3\images\train\img_2310.jpg
250 работаем  с файлом:  ..\avto\train_data3\images\train\img_2313.jpg
251 работаем  с файлом:  ..\avto\train_data3\images\train\img_2316.jpg
252 работаем  с файлом:  ..\avto\train_data3\images\train\img_2319.jpg
253 работаем  с файлом:  ..\avto\train_data3\images\train\img_2320.jpg
254 работаем  с файлом:  ..\avto\train_data3\images\train\img_2321.jpg
255 работаем  с файлом:  ..\avto\train_data3\images\train\img_2322.jpg
256 работаем  с файлом:  ..\avto\train_data3\images\train\img_2327.jpg
257 работаем  с файлом:  ..\avto\train_data3\images\train\img_2330.jpg
258 работаем  с файлом:  ..\avto\train_data3\images\train\img_2331.jpg
259 ра

361 работаем  с файлом:  ..\avto\train_data3\images\train\img_2568.jpg
362 работаем  с файлом:  ..\avto\train_data3\images\train\img_2570.jpg
363 работаем  с файлом:  ..\avto\train_data3\images\train\img_2574.jpg
364 работаем  с файлом:  ..\avto\train_data3\images\train\img_2575.jpg
365 работаем  с файлом:  ..\avto\train_data3\images\train\img_2577.jpg
366 работаем  с файлом:  ..\avto\train_data3\images\train\img_2578.jpg
367 работаем  с файлом:  ..\avto\train_data3\images\train\img_2579.jpg
368 работаем  с файлом:  ..\avto\train_data3\images\train\img_2581.jpg
369 работаем  с файлом:  ..\avto\train_data3\images\train\img_2583.jpg
370 работаем  с файлом:  ..\avto\train_data3\images\train\img_2584.jpg
371 работаем  с файлом:  ..\avto\train_data3\images\train\img_2586.jpg
372 работаем  с файлом:  ..\avto\train_data3\images\train\img_2588.jpg
373 работаем  с файлом:  ..\avto\train_data3\images\train\img_2590.jpg
374 работаем  с файлом:  ..\avto\train_data3\images\train\img_2593.jpg
375 ра

475 работаем  с файлом:  ..\avto\train_data3\images\train\img_2733_heic.jpg
476 работаем  с файлом:  ..\avto\train_data3\images\train\img_2734_heic.jpg
477 работаем  с файлом:  ..\avto\train_data3\images\train\img_2735.jpg
478 работаем  с файлом:  ..\avto\train_data3\images\train\img_2735_heic.jpg
479 работаем  с файлом:  ..\avto\train_data3\images\train\img_2737.jpg
480 работаем  с файлом:  ..\avto\train_data3\images\train\img_2737_heic.jpg
481 работаем  с файлом:  ..\avto\train_data3\images\train\img_2738_heic.jpg
482 работаем  с файлом:  ..\avto\train_data3\images\train\img_2739.jpg
483 работаем  с файлом:  ..\avto\train_data3\images\train\img_2740_heic.jpg
484 работаем  с файлом:  ..\avto\train_data3\images\train\img_2741.jpg
485 работаем  с файлом:  ..\avto\train_data3\images\train\img_2742.jpg
486 работаем  с файлом:  ..\avto\train_data3\images\train\img_2743.jpg
487 работаем  с файлом:  ..\avto\train_data3\images\train\img_2743_heic.jpg
488 работаем  с файлом:  ..\avto\train_dat

In [59]:
d = file_recognition('..\\avto\\start_dataset\\test\\img_2717.jpg')
print(d)

6
повернули
сохранили поворот Y:\img_2717.jpg
ФАЙЛ НЕ РАСПОЗНАН ..\avto\start_dataset\test\img_2717.jpg
0.0


In [63]:
img = Image.open('..\\avto\\start_dataset\\test\\img_2717.jpg')
img = img.rotate(-90, expand=True)
img.save('Y:\\img_2717.jpg')

In [15]:
from exif import Image as exif

In [ ]:
#не распознанные img_2674.heic img_2715.jpg img_2717.jpg

In [40]:
get_focus_from_exif("..\\avto\\start_dataset\\test\\img_1602.jpg")

17

In [29]:
with open("..\\avto\\start_dataset\\test\\img_1599.jpg", "rb") as palm_1_file:
    palm_1_image = exif(palm_1_file)
with open("..\\avto\\start_dataset\\test\\img_1602.jpg", "rb") as palm_2_file:
    palm_2_image = exif(palm_2_file)
images = [palm_1_image, palm_2_image]

In [24]:
if palm_1_image.has_exif:
    status = f"contains EXIF (version {palm_1_image.exif_version}) information."
else:
    status = "does not contain any EXIF information."
print(f"Image {status}")

Image contains EXIF (version 0232) information.


In [30]:
image_members = []

for image in images:
    image_members.append(dir(image))

for index, image_member_list in enumerate(image_members):
    print(f"Image {index} contains {len(image_member_list)} members:")
    print(f"{image_member_list}\n")

Image 0 contains 74 members:
['<unknown EXIF tag 316>', '<unknown EXIF tag 42080>', '_exif_ifd_pointer', '_gps_ifd_pointer', '_segments', 'aperture_value', 'brightness_value', 'color_space', 'components_configuration', 'compression', 'datetime', 'datetime_digitized', 'datetime_original', 'delete', 'delete_all', 'digital_zoom_ratio', 'exif_version', 'exposure_bias_value', 'exposure_mode', 'exposure_program', 'exposure_time', 'f_number', 'flash', 'flashpix_version', 'focal_length', 'focal_length_in_35mm_film', 'get', 'get_all', 'get_file', 'get_thumbnail', 'gps_altitude', 'gps_altitude_ref', 'gps_dest_bearing', 'gps_dest_bearing_ref', 'gps_horizontal_positioning_error', 'gps_img_direction', 'gps_img_direction_ref', 'gps_latitude', 'gps_latitude_ref', 'gps_longitude', 'gps_longitude_ref', 'gps_speed', 'gps_speed_ref', 'has_exif', 'jpeg_interchange_format', 'jpeg_interchange_format_length', 'lens_make', 'lens_model', 'lens_specification', 'list_all', 'make', 'maker_note', 'metering_mode', 

In [41]:
palm_1_image.get('focal_length_in_35mm_film', 'Unknown')

17

In [43]:
for i in image_member_list:
    print(i,palm_1_image.get(i, 'Unknown'))

<unknown EXIF tag 316> Unknown
<unknown EXIF tag 42080> Unknown
_exif_ifd_pointer 226
_gps_ifd_pointer 2192
_segments Unknown
aperture_value 2.5260688112781806
brightness_value 8.91151083143064
color_space ColorSpace.UNCALIBRATED
components_configuration Unknown
compression 6
datetime 2022:07:07 11:04:56
datetime_digitized 2022:07:07 11:04:56
datetime_original 2022:07:07 11:04:56
delete Unknown
delete_all Unknown
digital_zoom_ratio 1.261050875729775
exif_version 0232
exposure_bias_value 0.0
exposure_mode ExposureMode.AUTO_EXPOSURE
exposure_program ExposureProgram.NORMAL_PROGRAM
exposure_time 0.0027548209366391185
f_number 2.4
flash Flash(flash_fired=False, flash_return=FlashReturn.NO_STROBE_RETURN_DETECTION_FUNCTION, flash_mode=FlashMode.COMPULSORY_FLASH_SUPPRESSION, flash_function_not_present=False, red_eye_reduction_supported=False, reserved=0)
flashpix_version Unknown
focal_length 1.54
focal_length_in_35mm_film 17
get Unknown
get_all Unknown
get_file Unknown
get_thumbnail Unknown
gp

In [34]:
for index, image in enumerate(images):
    print(f"Lens and OS - Image {index}")
    print("---------------------")
    print(f"focal_length_in_35mm_film: {image.get('focal_length_in_35mm_film', 'Unknown')}")
    print(f"Lens model: {image.get('lens_model', 'Unknown')}")
    print(f"Lens specification: {image.get('lens_specification', 'Unknown')}")
    print(f"OS version: {image.get('software', 'Unknown')}\n")

Lens and OS - Image 0
---------------------
focal_length_in_35mm_film: 17
Lens model: iPhone 11 back dual wide camera 1.54mm f/2.4
Lens specification: (1.5399999618512084, 4.25, 1.8, 2.4)
OS version: 15.5

Lens and OS - Image 1
---------------------
focal_length_in_35mm_film: 17
Lens model: iPhone 11 back dual wide camera 1.54mm f/2.4
Lens specification: (1.5399999618512084, 4.25, 1.8, 2.4)
OS version: 15.5



In [31]:
for i in image_member_list:
    print(i)

<unknown EXIF tag 316>
<unknown EXIF tag 42080>
_exif_ifd_pointer
_gps_ifd_pointer
_segments
aperture_value
brightness_value
color_space
components_configuration
compression
datetime
datetime_digitized
datetime_original
delete
delete_all
digital_zoom_ratio
exif_version
exposure_bias_value
exposure_mode
exposure_program
exposure_time
f_number
flash
flashpix_version
focal_length
focal_length_in_35mm_film
get
get_all
get_file
get_thumbnail
gps_altitude
gps_altitude_ref
gps_dest_bearing
gps_dest_bearing_ref
gps_horizontal_positioning_error
gps_img_direction
gps_img_direction_ref
gps_latitude
gps_latitude_ref
gps_longitude
gps_longitude_ref
gps_speed
gps_speed_ref
has_exif
jpeg_interchange_format
jpeg_interchange_format_length
lens_make
lens_model
lens_specification
list_all
make
maker_note
metering_mode
model
offset_time
offset_time_digitized
offset_time_original
orientation
photographic_sensitivity
pixel_x_dimension
pixel_y_dimension
resolution_unit
scene_capture_type
scene_type
sensing_m

In [97]:
import exifread
f = open('..\\avto\\start_dataset\\test\\img_2674.heic', 'rb')
#f = open('..\\avto\\start_dataset\\test\\img_1598.jpg', 'rb')

tags = exifread.process_file(f,details=False)

NoParser: hdlr

In [96]:
tags['EXIF FocalLengthIn35mmFilm']

(0xA405) Short=17 @ 608

In [93]:
for tag in tags.keys():
    print ("Key: %s, value %s" % (tag, tags[tag]))

Key: Image Make, value Apple
Key: Image Model, value iPhone 11
Key: Image Orientation, value Horizontal (normal)
Key: Image XResolution, value 72
Key: Image YResolution, value 72
Key: Image ResolutionUnit, value Pixels/Inch
Key: Image Software, value 15.5
Key: Image DateTime, value 2022:07:07 11:04:10
Key: Image HostComputer, value iPhone 11
Key: Image YCbCrPositioning, value Centered
Key: Image ExifOffset, value 226
Key: GPS GPSLatitudeRef, value N
Key: GPS GPSLatitude, value [54, 19, 4599/100]
Key: GPS GPSLongitudeRef, value E
Key: GPS GPSLongitude, value [48, 23, 861/20]
Key: GPS GPSAltitudeRef, value 0
Key: GPS GPSAltitude, value 475297/2619
Key: GPS GPSSpeedRef, value K
Key: GPS GPSSpeed, value 0
Key: GPS GPSImgDirectionRef, value T
Key: GPS GPSImgDirection, value 390347/2216
Key: GPS GPSDestBearingRef, value T
Key: GPS GPSDestBearing, value 390347/2216
Key: GPS Tag 0x001F, value 54304/11167
Key: Image GPSInfo, value 2192
Key: Thumbnail Compression, value JPEG (old-style)
Key: Thu

In [73]:
decodeImage('..\avto\start_dataset\test\img_2674.heic')

NameError: name 'pyheif' is not defined

In [103]:
heif_file = pillow_heif.open_heif('..\\avto\\start_dataset\\test\\img_2674.heic')
print("image mode:", heif_file.mode)
print("image data length:", len(heif_file.data))
print("image data stride:", heif_file.stride)
heif_file.convert_to("RGB;16")  # convert 10 bit image to RGB 16 bit.
print("image mode:", heif_file.info.keys())

image mode: RGB
image data length: 36578304
image data stride: 12096
image mode: dict_keys(['exif', 'xmp', 'primary', 'metadata', 'icc_profile', 'icc_profile_type'])


In [108]:
print(heif_file.info['exif'])

b'Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x0b\x01\x0f\x00\x02\x00\x00\x00\x06\x00\x00\x00\x92\x01\x10\x00\x02\x00\x00\x00\n\x00\x00\x00\x98\x01\x12\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x00\xa2\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x00\xaa\x01(\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x011\x00\x02\x00\x00\x00\x07\x00\x00\x00\xb2\x012\x00\x02\x00\x00\x00\x14\x00\x00\x00\xba\x01<\x00\x02\x00\x00\x00\n\x00\x00\x00\xce\x87i\x00\x04\x00\x00\x00\x01\x00\x00\x00\xd8\x88%\x00\x04\x00\x00\x00\x01\x00\x00\t\x1a\x00\x00\x00\x00Apple\x00iPhone 12\x00\x00\x00\x00H\x00\x00\x00\x01\x00\x00\x00H\x00\x00\x00\x0115.4.1\x00\x002022:07:10 21:12:03\x00iPhone 12\x00\x00$\x82\x9a\x00\x05\x00\x00\x00\x01\x00\x00\x02\x8e\x82\x9d\x00\x05\x00\x00\x00\x01\x00\x00\x02\x96\x88"\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x88\'\x00\x03\x00\x00\x00\x01\x03\xe8\x00\x00\x90\x00\x00\x07\x00\x00\x00\x040232\x90\x03\x00\x02\x00\x00\x00\x14\x00\x00\x02\x9e\x90\x04\x00\x02\x00\x00\x00\

In [ ]:
 heif_file = pillow_heif.read_heif(file)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,

In [ ]:
https://pillow-heif.readthedocs.io/en/latest/reference/API.html